In [1]:
import glob
import os
import random
import shutil
import sys
from pathlib import Path

import numpy as np
import scipy.io
import torch

current_dir = os.path.dirname(os.path.abspath("__file__"))  # 当前目录
parent_dir = os.path.abspath(os.path.join(current_dir, ".."))  # 上一级目录
sys.path.append(parent_dir)
grandparent_dir = os.path.abspath(os.path.join(parent_dir, ".."))  # 上两级目录
sys.path.append(grandparent_dir)

import data_function
import metrics
import tools
import utils

# from test_case.ex_main.model import *
# from test_case.ex_main.train_dataloader import getData
from ex_main.model import *
from ex_main.train_dataloader import getData
from utils.config_plot import *

In [2]:
ppath = "/home/hy/project/reid/ex_main/"

# Config
config_file_path = os.path.join(ppath, "config.py")
config = utils.common.read_config_file(config_file_path)

# Initialize a logger tool
logger = utils.logger.Logger(config.outputs_path)
logger.info("#" * 50)
logger.info(f"Task: {config.taskname}")
logger.info(f"Using device: {config.device}")
logger.info(f"Using data type: {config.dtype}")

# Set environment
random.seed(config.seed)
np.random.seed(config.seed)
torch.manual_seed(config.seed)
torch.cuda.manual_seed(config.seed)
torch.cuda.manual_seed_all(config.seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False  # The result cannot be reproduced when True

vis_outputs_path = "./results/vis/"
if os.path.exists(vis_outputs_path):
    shutil.rmtree(vis_outputs_path)

2024-03-20 09:28:38 INFO ##################################################
2024-03-20 09:28:38 INFO Task: ReID_Task
2024-03-20 09:28:38 INFO Using device: cuda
2024-03-20 09:28:38 INFO Using data type: torch.float32


In [3]:
# config.device = "cpu"
config.test_batch_size = 512
_, query_loader, gallery_loader, num_classes = getData(config=config)

model = ReidNet(num_classes=num_classes, config=config, logger=logger).to(config.device)
path = os.path.join(ppath, "outputs/models/model_120.tar")
utils.network.load_network(model, path, config.device)
print("Model numbers of parameters: {}".format(utils.network.count_parameters(model)))

successfully load imagenet pre-trained resnet50-ibn model
Successfully loaded pretrained weights from "/home/hy/project/reid/ex_main/outputs/models/model_120.tar"
Model numbers of parameters: 34989120


In [4]:
# CMC, mAP = metrics.test_function(
#     model, query_loader, gallery_loader, config=config, logger=logger
# )
# message = (
#     "Testing: dataset_name: {} top1:{:.4f} top5:{:.4f} top10:{:.4f} mAP:{:.4f}"
# ).format(config.dataset_name, CMC[0], CMC[4], CMC[9], mAP)
# logger.info(message)

print("Loading features ...")
qf = torch.load(os.path.join(ppath, "outputs/temps/query_features_market1501.pt"))
gf = torch.load(os.path.join(ppath, "outputs/temps/gallery_features_market1501.pt"))
print(qf.shape, gf.shape)

print("Normalzing features with L2 norm ...")
qf = F.normalize(qf, p=2, dim=1)
gf = F.normalize(gf, p=2, dim=1)

distmat, rank_results = metrics.distance.compute_distance_matrix(qf, gf)

Loading features ...
torch.Size([3368, 2048]) torch.Size([15913, 2048])
Normalzing features with L2 norm ...


In [5]:
dataset = data_function.datasets.Market1501(root=config.dataset_path)

# tools.visualize_ranked_results.visualize_ranked_results(
#     distmat,
#     [dataset.query, dataset.gallery],
#     "image",
#     width=128,
#     height=256,
#     save_dir=os.path.join(vis_outputs_path, "ranked_results"),
#     topk=10,
# )

In [6]:
def val_collate_fn(batch):
    imgs, pids, camids, impath = zip(*batch)
    return torch.stack(imgs, dim=0), pids, camids, impath


hmap_loader = query_loader
hmap_loader.collate_fn = val_collate_fn
tools.visualize_actmap.visactmap(
    model,
    hmap_loader,
    os.path.join(vis_outputs_path, "hmap"),
    width=config.img_width,
    height=config.img_height,
    use_gpu=True,
)

Visualizing activation maps for dataset ...
